<a href="https://colab.research.google.com/github/Iliyyin/ml-dl/blob/main/Examining_Effects_Of_Bottlenecks_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import time
from PIL import Image
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision import transforms
import torch
import torch.nn as nn

In [2]:
class BottleneckImageFolder(ImageFolder):
    def __getitem__(self, index):
        path, target = self.samples[index]
        #Bottleneck1: Synchronous I/O.
        image = Image.open(path).convert('RGB')
        #Bottleneck2: Simulated CPU delay.
        time.sleep(0.001)
        if self.transform is not None:
            image = self.transform(image)
        if self.target_transform is not None:
            target = self.target_transform(target)
        return image, torch.tensor(target, dtype = torch.long)

In [3]:
if __name__ == '__main__':
  transform = transforms.Compose([
    transforms.Resize((224, 224)), transforms.ToTensor(), ])

In [5]:
import os
import shutil

# Create a subdirectory for the images
class_dir = '/content/Images/all_images'
os.makedirs(class_dir, exist_ok=True)

# Move all image files into the subdirectory
for filename in os.listdir('/content/Images'):
    if filename.endswith('.jpg'):
        shutil.move(os.path.join('/content/Images', filename), class_dir)

print(f"Moved all images to {class_dir}")

Moved all images to /content/Images/all_images


In [8]:
loader = DataLoader(dataset, batch_size = 32, num_workers = 0)

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_adaptive = nn.Sequential(
    nn.Conv2d(3, 16, kernel_size = 3, stride = 1, padding = 1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size = 2, stride = 2),
    nn.AdaptiveAvgPool2d(output_size = (1, 1)),
    nn.Flatten(),
    nn.Linear(16, 10)
).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_adaptive.parameters(), lr = 0.001)

In [19]:
for epoch in range(6):
  epoch_start_time = time.time()
  with torch.profiler.profile(
      activities = [torch.profiler.ProfilerActivity.CPU],
      schedule = torch.profiler.schedule(wait = 1, warmup = 1, active = 3, repeat = 2),
      on_trace_ready=torch.profiler.tensorboard_trace_handler("./logs/baseline", worker_name="worker0"),
      record_shapes = True,
      profile_memory = True,
      with_stack = True
  ) as prof:
      for i, (inputs, labels) in enumerate(loader):
          inputs, labels = inputs.to(device), labels.to(device)
          optimizer.zero_grad()
          outputs = model_adaptive(inputs)
          loss = criterion(outputs, labels)
          loss.backward()
          optimizer.step()
          prof.step()
epoch_time = time.time() - epoch_start_time
print(f"Epoch {epoch + 1} completed in {epoch_time:.2f} seconds")
print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=10))


Epoch 6 completed in 10.66 seconds
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                          ProfilerStep*         1.49%       7.549ms       100.00%     508.299ms     254.149ms      -9.18 MB     -79.63 MB             2  
                                       aten::max_pool2d         0.00%      24.918us        19.50%      99.132ms      99.132ms      36.75 MB           0 B             1  
                          aten::max_pool2d_with_indices        19.50%      99.107ms        19.50%      99.107ms    